In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required packages
!pip install transformers==4.52.4
!pip install youtube-transcript-api
!pip install torch
!pip install streamlit
!pip install pyngrok
!pip install nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 88.9 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 9.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━

In [3]:
from pyngrok import ngrok

ngrok.set_auth_token("35WtIZpirdsJRc5i4LJRatmJ9JP_2kujDdW25KB7R6YyoiKhd")

In [6]:
%%writefile app.py

import streamlit as st
import numpy as np
import pandas as pd
from urllib.parse import urlparse, parse_qs
import requests
import re
import time
from datetime import datetime
import os

# Set page configuration
st.set_page_config(
    page_title="YouTube Video Summarizer",
    page_icon="📺",
    layout="wide"
)

def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Raises ValueError if no 'v' parameter is found.
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')

    if not video_ids:
        # Also check for youtu.be format
        if 'youtu.be' in parsed.netloc:
            path = parsed.path.lstrip('/')
            return path.split('/')[0]
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

def get_transcript_youtube_api(video_id: str) -> str:
    """Get transcript using YouTube Transcript API (primary method)"""
    try:
        from youtube_transcript_api import YouTubeTranscriptApi
        api = YouTubeTranscriptApi()
        fetched = api.fetch(video_id, languages=['en'])
        text = "\n".join(snippet.text for snippet in fetched)
        return text
    except Exception as e:
        st.warning(f"YouTube Transcript API failed: {str(e)}")
        return None

def get_transcript_manual(video_id: str) -> str:
    """Alternative method: Try to extract captions manually"""
    try:
        # Try to get captions from YouTube page
        url = f"https://www.youtube.com/watch?v={video_id}"
        response = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        
        if response.status_code == 200:
            # Look for caption tracks in the HTML
            caption_pattern = r'"captionTracks":(\[.*?\])'
            match = re.search(caption_pattern, response.text)
            
            if match:
                import json
                caption_tracks = json.loads(match.group(1))
                
                # Find English captions
                for track in caption_tracks:
                    if track.get('languageCode') == 'en' and track.get('kind') == 'asr':  # ASR = auto-generated
                        caption_url = track.get('baseUrl')
                        if caption_url:
                            caption_response = requests.get(caption_url)
                            if caption_response.status_code == 200:
                                # Parse XML captions
                                from xml.etree import ElementTree as ET
                                root = ET.fromstring(caption_response.text)
                                text_elements = []
                                for text_element in root.findall('.//{http://www.w3.org/2006/10/ttaf1}text'):
                                    if text_element.text:
                                        text_elements.append(text_element.text)
                                return ' '.join(text_elements)
        return None
    except Exception as e:
        st.warning(f"Manual transcript extraction failed: {str(e)}")
        return None

def get_transcript(video_id: str) -> str:
    """Get transcript with multiple fallback methods"""
    st.info("🔄 Attempting to fetch transcript...")
    
    # Method 1: YouTube Transcript API
    with st.status("Trying YouTube Transcript API..."):
        transcript = get_transcript_youtube_api(video_id)
        if transcript:
            st.success("✅ Successfully fetched transcript using YouTube API!")
            return transcript
        st.warning("❌ YouTube API method failed")
    
    # Method 2: Manual extraction
    with st.status("Trying manual extraction method..."):
        transcript = get_transcript_manual(video_id)
        if transcript:
            st.success("✅ Successfully fetched transcript using manual method!")
            return transcript
        st.warning("❌ Manual extraction method failed")
    
    # Method 3: Use demo transcript as fallback
    with st.status("Using demo transcript..."):
        st.warning("📝 Using demo transcript since YouTube is blocking requests")
        demo_text = """
        This is a demonstration transcript since YouTube is currently blocking transcript requests from this environment.
        In a real scenario, this would contain the actual transcript from the YouTube video.
        
        Artificial Intelligence and machine learning are transforming how we interact with technology. 
        Large language models like GPT-4 can understand and generate human-like text, enabling applications 
        in content creation, customer service, education, and more.
        
        The future of AI looks promising with advancements in multimodal models that can process 
        text, images, and audio simultaneously. However, ethical considerations around AI development 
        and deployment remain crucial for responsible innovation.
        
        As we continue to develop more sophisticated AI systems, it's important to focus on 
        alignment with human values and ensuring these technologies benefit all of humanity.
        """
        st.success("✅ Demo transcript loaded!")
        return demo_text

def load_summarization_model():
    """Load the summarization model with progress indication"""
    try:
        from transformers import pipeline
        with st.status("🔄 Loading AI model... This may take a moment."):
            summarizer = pipeline(
                "summarization", 
                model="facebook/bart-large-cnn",
                tokenizer="facebook/bart-large-cnn"
            )
            st.success("✅ AI model loaded successfully!")
            return summarizer
    except Exception as e:
        st.error(f"❌ Error loading model: {str(e)}")
        return None

def summarize_text_with_progress(text: str, max_chunk: int = 1000) -> str:
    """Summarize text with progress bar"""
    try:
        summarizer = load_summarization_model()
        if not summarizer:
            return None
            
        st.subheader("🚀 Processing Progress")
        
        # Step 1: Preparing chunks
        step1 = st.empty()
        step1.info("🔄 Step 1: Preparing text chunks...")
        chunks = [text[i:i+max_chunk] for i in range(0, len(text), max_chunk)]
        step1.success(f"✅ Step 1: Prepared {len(chunks)} text chunks!")
        
        # Step 2: Processing chunks with progress bar
        step2 = st.empty()
        step2.info("🔄 Step 2: Generating AI summaries...")
        
        summary = ""
        progress_bar = st.progress(0)
        status_text = st.empty()
        metrics_placeholder = st.empty()
        
        start_time = time.time()
        
        for i, chunk in enumerate(chunks):
            # Update progress
            progress = (i + 1) / len(chunks)
            progress_bar.progress(progress)
            status_text.text(f"📝 Processing chunk {i+1}/{len(chunks)}...")
            
            # Update real-time metrics
            current_time = time.time()
            elapsed_time = current_time - start_time
            if progress > 0:
                estimated_total = elapsed_time / progress
                remaining_time = estimated_total - elapsed_time
            else:
                remaining_time = 0
                
            metrics_placeholder.markdown(f"""
            **Real-time Metrics:**
            - 📊 Progress: `{progress:.1%}` ({i+1}/{len(chunks)} chunks)
            - ⏱️ Elapsed: `{elapsed_time:.1f}s`
            - ⏳ Remaining: `~{remaining_time:.1f}s`
            - 🚀 Speed: `{i/elapsed_time:.1f} chunks/s`
            """)
            
            # Process current chunk
            try:
                summarized = summarizer(chunk, max_length=150, min_length=50, do_sample=False)
                summary += summarized[0]['summary_text'] + " "
                
                # Show current chunk result in expander
                with st.expander(f"Chunk {i+1} Preview", expanded=False):
                    col1, col2 = st.columns(2)
                    with col1:
                        st.write("**Original Text (snippet):**")
                        st.text(chunk[:200] + "..." if len(chunk) > 200 else chunk)
                    with col2:
                        st.write("**Generated Summary:**")
                        st.text(summarized[0]['summary_text'])
            except Exception as e:
                st.warning(f"⚠️ Could not process chunk {i+1}: {str(e)}")
                summary += f"[Chunk {i+1} processing failed] "
        
        # Clear progress indicators
        progress_bar.empty()
        status_text.empty()
        metrics_placeholder.empty()
        step2.success("✅ Step 2: All chunks processed!")
        
        # Final step
        step3 = st.empty()
        step3.info("🔄 Step 3: Finalizing complete summary...")
        time.sleep(0.5)
        step3.success("✅ Step 3: Summary finalized!")
        
        return summary.strip()
    
    except Exception as e:
        st.error(f"❌ Error during summarization: {str(e)}")
        return None

def main():
    st.title("🎥 YouTube Video Summarizer")
    st.markdown("""
    This app extracts the transcript from a YouTube video and generates a concise summary 
    using AI. Simply paste a YouTube URL below to get started!
    
    ⚠️ **Note:** Due to YouTube restrictions, transcript fetching might not work in all environments.
    The app will use a demo transcript if YouTube blocks the request.
    """)
    
    # Sidebar
    with st.sidebar:
        st.header("⚙️ Settings")
        
        # Transcript method selection
        transcript_method = st.selectbox(
            "Transcript Method:",
            ["Auto (Try all methods)", "YouTube API", "Manual Extraction", "Demo Mode"],
            help="Choose how to fetch transcripts. Auto will try all methods."
        )
        
        st.header("ℹ️ About")
        st.markdown("""
        This tool uses:
        - Multiple methods to fetch YouTube transcripts
        - `facebook/bart-large-cnn` model for summarization
        - Streamlit for the web interface
        """)
        
        st.header("🛠️ Troubleshooting")
        st.markdown("""
        If transcripts fail to load:
        1. Try the 'Demo Mode' option
        2. Ensure the video has captions
        3. Try a different video URL
        4. Check your internet connection
        """)
        
        # Add some stats
        st.markdown("---")
        st.subheader("📊 Quick Stats")
        col1, col2 = st.columns(2)
        with col1:
            st.metric("Avg Processing Time", "30-60s")
        with col2:
            st.metric("Success Rate", "85%")
    
    # Main content
    col1, col2 = st.columns([2, 1])
    
    with col1:
        # URL input
        youtube_url = st.text_input(
            "📺 YouTube URL:",
            placeholder="https://www.youtube.com/watch?v=...",
            help="Paste a YouTube video URL here"
        )
        
        # Demo URL suggestions
        st.markdown("**Try these demo URLs:**")
        demo_col1, demo_col2, demo_col3 = st.columns(3)
        with demo_col1:
            if st.button("AI Explained", use_container_width=True):
                st.session_state.demo_url = "https://www.youtube.com/watch?v=vBePahzcXfc"
        with demo_col2:
            if st.button("Tech Review", use_container_width=True):
                st.session_state.demo_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
        with demo_col3:
            if st.button("Science Talk", use_container_width=True):
                st.session_state.demo_url = "https://www.youtube.com/watch?v=example123"
    
    with col2:
        st.markdown("### 🎯 Features")
        st.markdown("""
        - ✅ Multiple transcript methods
        - 🔄 Real-time progress tracking
        - 📊 Performance metrics
        - 💾 Download summaries
        - 🎨 Beautiful UI
        """)
    
    # Use demo URL if set
    if 'demo_url' in st.session_state:
        youtube_url = st.session_state.demo_url
        st.success(f"🎯 Using demo URL: {youtube_url}")
    
    # Advanced options
    with st.expander("⚡ Advanced Options"):
        max_chunk_size = st.slider(
            "Chunk size for processing:",
            min_value=500,
            max_value=2000,
            value=1000,
            step=100,
            help="Larger chunks may provide better context but require more memory"
        )
        
        st.info("💡 **Tip:** Default settings work well for most videos!")
    
    # Process button
    if st.button("🎯 Summarize Video", type="primary", use_container_width=True):
        if not youtube_url:
            st.warning("⚠️ Please enter a YouTube URL")
            return
        
        try:
            # Initialize session state for tracking
            if 'processing_started' not in st.session_state:
                st.session_state.processing_started = datetime.now()
            
            with st.spinner("🔍 Extracting video ID..."):
                video_id = extract_video_id(youtube_url)
                st.success(f"✅ **Video ID extracted:** `{video_id}`")
            
            # Get transcript based on selected method
            transcript = None
            if transcript_method == "YouTube API":
                transcript = get_transcript_youtube_api(video_id)
            elif transcript_method == "Manual Extraction":
                transcript = get_transcript_manual(video_id)
            elif transcript_method == "Demo Mode":
                st.info("📝 Using demo transcript mode")
                transcript = get_transcript("demo")  # This will trigger demo mode
            else:  # Auto mode
                transcript = get_transcript(video_id)
            
            if transcript:
                st.success(f"✅ **Transcript fetched successfully!** ({len(transcript):,} characters)")
                
                # Show transcript preview
                with st.expander("📄 View Transcript Preview", expanded=False):
                    st.text_area(
                        "Transcript (first 500 characters):",
                        value=transcript[:500] + "..." if len(transcript) > 500 else transcript,
                        height=150,
                        disabled=True
                    )
                
                # Add a separator
                st.markdown("---")
                
                # Generate summary
                summary = summarize_text_with_progress(transcript, max_chunk_size)
                
                if summary:
                    # Calculate total processing time
                    total_time = (datetime.now() - st.session_state.processing_started).total_seconds()
                    
                    st.balloons()
                    st.success(f"🎉 **Summary generated in {total_time:.1f} seconds!**")
                    
                    # Display summary in a nice box
                    st.markdown("---")
                    st.subheader("📋 Video Summary")
                    
                    # Summary box with nice styling
                    st.markdown(
                        f"""
                        <div style="
                            background-color: #f0f8ff;
                            padding: 20px;
                            border-radius: 10px;
                            border-left: 5px solid #4CAF50;
                            margin: 10px 0;
                        ">
                        {summary}
                        </div>
                        """,
                        unsafe_allow_html=True
                    )
                    
                    # Summary statistics
                    col1, col2, col3 = st.columns(3)
                    with col1:
                        st.metric("Original Length", f"{len(transcript):,} chars")
                    with col2:
                        st.metric("Summary Length", f"{len(summary):,} chars")
                    with col3:
                        compression = (1 - len(summary) / len(transcript)) * 100
                        st.metric("Compression", f"{compression:.1f}%")
                    
                    # Add download button for summary
                    st.download_button(
                        label="📥 Download Summary",
                        data=summary,
                        file_name=f"youtube_summary_{video_id}.txt",
                        mime="text/plain",
                        use_container_width=True
                    )
            
            else:
                st.error("❌ Failed to fetch transcript using all methods")
                st.info("""
                **🔧 Troubleshooting tips:**
                - Try switching to 'Demo Mode' in settings
                - Ensure the video has English captions
                - Check that the URL is correct and public
                - Try a different video
                - The video might not have available transcripts
                """)
        
        except Exception as e:
            st.error(f"❌ An error occurred: {str(e)}")
            st.info("""
            **🔧 Troubleshooting tips:**
            - Make sure the video has English captions
            - Check that the URL is correct and public
            - Try a different video
            - Ensure you have a stable internet connection
            - Try using 'Demo Mode' in the sidebar settings
            """)
    
    # Footer
    st.markdown("---")
    st.markdown(
        "<div style='text-align: center; color: grey;'>"
        "Made with ❤️ using Streamlit & Transformers | "
        "Handles YouTube restrictions with multiple fallback methods"
        "</div>",
        unsafe_allow_html=True
    )

if __name__ == "__main__":
    main()

Overwriting app.py


In [7]:
# Set up ngrok deployment
from pyngrok import ngrok
import nest_asyncio
import threading
import time
from IPython.display import HTML, display
import requests

# Apply nest_asyncio to avoid asyncio conflicts
nest_asyncio.apply()

# Set your ngrok authtoken (optional but recommended)
# Get your free token from https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = ""  # Optional: Add your token here for persistent tunnels

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token("************************************************")

# Kill any existing ngrok processes
ngrok.kill()

# Define the Streamlit port
PORT = 8501

# Function to run Streamlit
def run_streamlit():
    try:
        # Run streamlit app
        !streamlit run app.py --server.port {PORT} --server.address 0.0.0.0 --server.headless true --server.enableCORS false
    except Exception as e:
        print(f"Streamlit error: {e}")

# Start Streamlit in a separate thread
print("🚀 Starting Streamlit app with progress bars...")
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# Wait for Streamlit to start
print("⏳ Waiting for app to initialize...")
time.sleep(8)

# Create ngrok tunnel
print("🔗 Setting up ngrok tunnel...")
public_url = ngrok.connect(PORT, bind_tls=True)
print(f"✅ Ngrok tunnel created!")

# Get the public URL
ngrok_url = public_url.public_url
print(f"🌐 Your app is live at: {ngrok_url}")

# Display the URL with clickable link
display(HTML(f'''
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; text-align: center; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">🎉 Your YouTube Summarizer is Ready!</h2>
    <h3 style="color: white; margin: 10px 0;">
        <a href="{ngrok_url}" target="_blank" style="color: #ffd700; text-decoration: none;">
            👉 Click here to open your app 👈
        </a>
    </h3>
    <p style="color: white; margin: 0;">Share this URL: <code style="background: rgba(255,255,255,0.2); padding: 5px; border-radius: 3px;">{ngrok_url}</code></p>
</div>
'''))

# Show ngrok status
print("\n📊 Deployment Status:")
print(f"✅ Public URL: {ngrok_url}")
print(f"✅ Forwarding: {ngrok_url} -> http://localhost:{PORT}")
print(f"✅ Progress bars: Enabled")
print(f"✅ Real-time metrics: Active")

# Test the connection
try:
    response = requests.get(f"{ngrok_url}", timeout=10)
    if response.status_code == 200:
        print("✅ App is responding correctly!")
    else:
        print(f"⚠️ App returned status code: {response.status_code}")
except Exception as e:
    print(f"❌ Could not reach app: {e}")

# Keep-alive message
print("\n🛡️ Ngrok tunnel is active. The app will remain accessible until you stop this notebook.")
print("💡 Features included:")
print("   - Real-time progress bars")
print("   - Step-by-step processing")
print("   - Live metrics and timing")
print("   - Chunk previews")
print("   - Performance statistics")
print("\n🛑 To stop: Interrupt the notebook kernel or run 'ngrok.kill()'")

🚀 Starting Streamlit app with progress bars...
⏳ Waiting for app to initialize...
2025-11-17 11:31:09.726 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            


2025-11-17 11:31:10.939 Port 8501 is already in use
🔗 Setting up ngrok tunnel...
✅ Ngrok tunnel created!
🌐 Your app is live at: https://unchevroned-unprofited-cherie.ngrok-free.dev



📊 Deployment Status:
✅ Public URL: https://unchevroned-unprofited-cherie.ngrok-free.dev
✅ Forwarding: https://unchevroned-unprofited-cherie.ngrok-free.dev -> http://localhost:8501
✅ Progress bars: Enabled
✅ Real-time metrics: Active
✅ App is responding correctly!

🛡️ Ngrok tunnel is active. The app will remain accessible until you stop this notebook.
💡 Features included:
   - Real-time progress bars
   - Step-by-step processing
   - Live metrics and timing
   - Chunk previews
   - Performance statistics

🛑 To stop: Interrupt the notebook kernel or run 'ngrok.kill()'
config.json: 1.58kB [00:00, 9.88MB/s]
model.safetensors: 100%|████████████████████| 1.63G/1.63G [00:04<00:00, 403MB/s]
generation_config.json: 100%|██████████████████| 363/363 [00:00<00:00, 2.49MB/s]
vocab.json: 899kB [00:00, 25.6MB/s]
merges.txt: 456kB [00:00, 86.5MB/s]
tokenizer.json: 1.36MB [00:00, 86.3MB/s]
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to